In [78]:
import numpy as np
import pandas as pd

def init(prs, thresh):
    #importing and preprocessing
    from sklearn.model_selection import cross_val_score
    data = pd.read_csv(r'C:\Users\Viral Shanker\MLfile.csv')
    data.head()
    data["constant"] = 1
    data = data.drop(columns = ["maxscore"])
    data = data.drop(columns = ["age", 'created1.1', 'score1.1', 'age1.1', 'stories1.1', 'ratio1.1', 'coms1.1', 'created2.1', 'score2.1', 'age2.1', 'stories2.1', 'ratio2.1', 'coms2.1', 'created3.1', 'score3.1', 'age3.1', 'stories3.1', 'ratio3.1', 'coms3.1', 'created4.1', 'score4.1', 'age4.1', 'stories4.1', 'ratio4.1', 'coms4.1', 'created5.1', 'score5.1', 'age5.1', 'stories5.1', 'ratio5.1', 'coms5.1'])
    data["finalScore"] = data["61selfScore"]
    data["success"] = 0
    from sklearn.utils import shuffle
    #data = shuffle(data)
    #getHour and decide whether o not to shuffle
    import datetime
    def getHourIndex(pers, intercept = 37, slope = 34):
        return(intercept + 34*pers)

    threshL = [thresh]
    pers = prs
    ind = getHourIndex(pers)

    curr = 1
    for x in threshL:
        data.loc[data.finalScore > x, "success"] = curr
        curr += 1
    ##data.loc[data.finalScore > thresh, "success"] = 1
    X = data[data.columns[3:ind]]
    y = data["finalScore"]
    X = pd.concat([X, data["created_utc"].apply(lambda s: pd.Series({'day':datetime.datetime.utcfromtimestamp(s).strftime('%a')}))], axis=1)
    t = pd.get_dummies(X["day"])
    X = X.join(t)
    X = X.drop(columns = ["day"])
    X = X.apply(pd.to_numeric)
    return(X,y)

def generalTest(reg, thresh, trainX, trainy, testX, testy, verbose):
        reg.fit(trainX, trainy)
        if verbose:
            print("The Score is: ", reg.score(testX, testy))
        a = np.where(testy > thresh)[0].tolist()
        preds = reg.predict(testX)
        p = np.where(preds > thresh)[0].tolist()
        #print(a)
        #print(p)
        c = 0
        i = 0
        m = 0
        for e in p:
            if e not in a:
                if verbose:
                    print("Incorrect Prediction", e, "actual score:", testy.tolist()[e], "predicted score:", preds.tolist()[e])
                i += 1
            else:
                if verbose:
                    print("Correct Prediction", e ,"actual score:", testy.tolist()[e], "predicted score:", preds.tolist()[e])
                c += 1
        for e in a:
            if e not in p:
                if verbose:
                    print("Missed Prediction", e, "actual score:", testy.tolist()[e], "predicted score:", preds.tolist()[e])
                m += 1
        if verbose:
            print("Correct:", c)
            print("Inorrect:", i)
            print("Missed:", m)
        return(c,i,m)

def generalRegTest(thresh, seds, MasterReg, norm, X, y, verbose = False):
    #Our generalized regression testing 
    if norm is True:
        X = X.apply(lambda x: x/x.max(), axis=0)
    ccnt = 0
    icnt = 0
    mcnt = 0
    for seed in seds:
        #print('for seed', seed)
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.25, random_state=seed)
        tem = generalTest(MasterReg, thresh, X_train, y_train, X_test, y_test, verbose)
        ccnt += tem[0]
        icnt += tem[1]
        mcnt += tem[2]
    print("Correct:", ccnt)
    print("Inorrect:", icnt)
    print("Missed:", mcnt)

In [79]:
prs = 6
cutOff = 2000
seds = [312,2,14,425]
dat = init(prs, cutOff)
X = dat[0]
y = dat[1]

In [84]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
regr = RandomForestRegressor(max_depth=2, n_estimators=100)
regrC = RandomForestClassifier(max_depth=5, random_state=0, n_estimators=100, class_weight = "balanced")
regD = MLPRegressor(solver='lbfgs', hidden_layer_sizes=100,
                           max_iter=300, shuffle=False, random_state=9876,
                           activation='relu')
regGB = GradientBoostingRegressor(loss = "ls", n_estimators = 1000)
params = {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls', 'verbose': 0}
clf = GradientBoostingRegressor(**params)

In [85]:
generalRegTest(5000, seds, clf, False,X, y, False)

Correct: 34
Inorrect: 14
Missed: 30
